# 추천 시스템 프로그램

    - 목표 : 데이터를 입력하고 추천받기 버튼을 누르면 추천해주는 프로그램 제작
    - 데이터 : 저장 방식은 현재는 변수에만 저장, 추후에 DB나 외부파일에 저장하는 방식으로 변경
    - 최종 목표 : 웹 어플리케이션에 서비스화

## 추천 방법 (비효율적 추천방법)
 #### ㅇ수작업 추천 
    - 수작업 추천은 경험에 의거해서 추천하는 방식
    - 일일히 추천하기도 귀찮고 추천의 신뢰성이 떨어질 수 있음
 #### ㅇ인기도 순으로 추천
    - 가장 인기있는 순서대로 추천
    - 인기있는 것만 고른 사용자에게는 상대적으로 인기없는 요소들만 추천들어갈 가능성 존재

## 추천방법 (효율적 추천방법)
#### ㅇ유저 기반 추천
    - 나랑 비슷한 선택을 한 유저를 찾아서 그 유저가 고른 선택지 중 내가 고르지 않은 선택지를 추천해주는 방식
    - 아이템 개수가 적을때 효율적, 선택지가 많아지면 비효율적

#### ㅇ아이템 기반 추천
    - 내가 고른 아이템과 함께 골라진 아이템을 찾아서 추천
    - 선택지가 많을때 가장 효율적인 선택지

#### ㅇ코사인 유사도 기반 추천 알고리즘
    - 코사인 유사도란 특정 정보와 유사한 정보를 각도로 판단하는 방법.
    - 코사인 유사도가 높으면 높을수록 유사한 선택지로 판단 가능하며
    - 코사인 유사도가 낮을수록 반대되는 성향을 가진 선택지로 판단할 수 있다.

In [1]:
# GUI구현을 위한 tkinter 임포트
from tkinter import *

In [2]:
# 이외의 필요 라이브러리 임포트
import math, random
from collections import defaultdict, Counter
import numpy as np

In [3]:
# 윈도우창 만들기
root = Tk()

In [4]:
# 윈도우창 크기 및 창 제목 바꾸기(창 크기는 .place()를 활용하는 경우만 적용가능!)
root.title("추천 시스템")
root.configure(width="110m", height="80m")

In [5]:
# global 선언을 받을 전역변수들 만들기
# 유저번호와 선택한 목록 준비
users_interests = {}
unique_interests = []

# 변수에 명목상 아무 값이나 넣음.
select1 = ""
select2 = ""
select3 = ""
regionresult=""

In [6]:
# 테스트 데이터
def set_dummy():
    global users_interests, unique_interests
    users_interests = {
    "a":["우유", "요구르트", "도시락", "과일야채음료", "치즈", "오뎅", "햇반"],
    "b":["레토르트류", "빵", "주먹밥", "우유", "요구르트"],
    "c":["반찬류", "냉장간편식", "두유", "샌드위치류", "생수", "김밥"],
    "d":["김밥", "우유", "과일채소", "디저트", "즉석조리식품"],
    "e":["반찬류", "두부", "빵", "샐러드"],
    "f":["떡", "요구르트", "치즈", "김밥", "디저트", "두유"],
    "g":["우유", "과일야채음료", "도시락", "주먹밥"],
    "h":["요구르트", "도시락", "생수", "빵"],
    "i":["샐러드", "햄버거류", "김밥", "우유"],
    "j":["과일야채음료", "오뎅", "주먹밥", "즉석조리식품"],
    "k":["빵", "김밥", "요구르트"],
    "l":["도시락", "레토르트류", "우유", "생수"]
    }
    set_unique_interests()
    L1.delete(0, END)
    L2.delete(0, END)
    L3.delete(0, END)
    idx=0
    
    for user in users_interests:
        L1.insert(idx, user)
        idx += 1
    idx = 0
    
    for region in unique_interests:
        L2.insert(idx, region)
        idx += 1



### 함수 정리

In [7]:
# 사용자가 선택한 요소 전체 목록 출력
# sorted는 중복요소는 몇 개던 하나로 치환해 계산
def set_unique_interests():
    global unique_interests
    unique_interests = sorted(list({interest
                                 for user_interests in users_interests.values()
                                 for interest in user_interests }))

In [8]:
#  listbox를 클릭했을때 클릭된 요소를 감지해 변수에 저장
# listbox1
def onselect1(evt):
    global select1
    w = evt.widget
    index = w.curselection()
    value = w.get(index)
    select1 = value
    select_noselect()
    print(select1)
    
# listbox2
def onselect2(evt):
    global select2
    w = evt.widget
    index = w.curselection()
    value = w.get(index)
    select2 = value
    print(select2)

# listbox3
def onselect3(evt):
    global select3
    w = evt.widget
    index = w.curselection()
    value = w.get(index)
    select3 = value
    print(select3)
    

    
def add_select():
    for region in unique_interests:
        L2.insert(idx, region)
        idx += 1
    L3.insert(select2)
    select_noselect()
    
# def delete_select():
#     print('test')
    
# def delete_user():
#     print('test')

In [9]:
# 코사인 유사도 구하는 함수
# 코사인 유사도 측정, 방향성이 가까울수록 같은 성향을 가진 요소로 취급
def cosine_similarity(v, w):
    return np.dot(v, w) / math.sqrt(np.dot(v, v) * np.dot(w, w))

In [10]:
# 유저 베이스 추천
# 1. 유저별 선택/미선택 분류 리스트 만들기
def make_user_interest_vector(user_interests):
    # 아이템 총 갯수로 따졌을때 어떤 유저가 무슨 요소에 관심이 있다고 한 경우
    # (1번 유저 a는 ["우유", "요구르트", "도시락", "과일야채음료", "치즈", "오뎅", "햇반"]에 관심있음)
    # 1로, 그렇지 않다면 0으로 처리
    return [1 if interest in user_interests else 0
            for interest in unique_interests]

# unique_interests[키값] 에 해당하는 요소가 관심사에 있으면 1, 없으면 0인 자료 생성
def get_user_interest_matrix():
    user_interest_matrix = list(map(make_user_interest_vector, users_interests.values()))
    return user_interest_matrix


# 최종적으로 이 함수만 호출해도 위쪽 2개가 자동 호출됨!
# 코사인 유사도를 통해서 한 아이템이 선택되었을때 다른 아이템이 같이 선택되는 빈도를 통해 연관도 집계
def get_user_similarities():
    user_similarities = [[cosine_similarity(interest_vector_i, interest_vector_j)
                          for interest_vector_j in get_user_interest_matrix()]
                         for interest_vector_i in get_user_interest_matrix()]
    return user_similarities

def most_similar_users_to(user_id):
    pairs = [(other_user_id, similarity)                      # 유사도가 
             for other_user_id, similarity in                 # 0이 아니면 일단
                enumerate(get_user_similarities()[user_id])         # 분류 대상으로 삼는
             if user_id != other_user_id and similarity > 0]  # 함수

    return sorted(pairs,                                      # 분류가 된 함수를 대상으로
                  key=lambda pair: pair[1],                   # 유사도가 높은것부터
                  reverse=True)                               # 정렬해줌


# 모든 유사도를 더했을때 가장 높은 유사도 총합을 갖는 유저의 자료를 추천해주는 함수
# include_current_interests 파라미터가 True인 경우는 이미 관심사인 것들까지 유사도를 더하는데 활용함.
def user_based_suggestions(include_current_interests=False):
    # 요소별 유사도 총합 구하기
    global regionresult
    suggestions = defaultdict(float)
    user_idx = list(users_interests).index(select1)
    for other_user_id, similarity in most_similar_users_to(user_idx):
        for interest in users_interests[list(users_interests)[other_user_id]]:
            suggestions[interest] += similarity

    # 정렬시킨 리스트로 바꿈
    suggestions = sorted(suggestions.items(),
                         key=lambda pair: pair[1],
                         reverse=True)

    # 이미 관심사로 표시한것은 제외
    if include_current_interests:
        regionresult = suggestions
        print(regionresult)
    else:
        regionresult = [(suggestion, weight)
                for suggestion, weight in suggestions
                if suggestion not in users_interests[select1]]
        print(regionresult)
        

In [11]:
# 아이템 베이스 추천
# 아이템 기반(유저가 선택한 아이템과 비슷한 아이템 추천)
#

# 하나의 아이템에 대해 관심을 표명한 사용자는 1로, 그렇지 않은 사용자는 0으로 표시
# [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]는 0번 자료가 '과일채소'이므로 5번유저가 이 키워드에 관심을 표명했음을 나타냄
def get_interest_user_matrix():
    interest_user_matrix = [[user_interest_vector[j]
                             for user_interest_vector in get_user_interest_matrix()]
                            for j, _ in enumerate(unique_interests)]
    return interest_user_matrix


# 아이템 간 코사인 유사도 적용. 이 아이템에 관심있는 사람이 나머지 아이템에도 관심이 있는가 없는가
def get_interest_similarities():
    interest_similarities = [[cosine_similarity(user_vector_i, user_vector_j)
                              for user_vector_j in get_interest_user_matrix()]
                             for user_vector_i in get_interest_user_matrix()]
    return interest_similarities

# 특정 아이템을 선택하면 그 아이템과 함께 가장 많이 선택된 다음 아이템 선택
def most_similar_interests_to(interest_id):
    similarities = get_interest_similarities()[interest_id]
    pairs = [(unique_interests[other_interest_id], similarity)
             for other_interest_id, similarity in enumerate(similarities)
             if interest_id != other_interest_id and similarity > 0]
    return sorted(pairs,
                  key=lambda pair: pair[1],
                  reverse=True)


# 역시 비슷한 관심사를 가진것들끼리 더함
def item_based_suggestions(include_current_interests=False):
    suggestions = defaultdict(float)
    user_idx = list(users_interests).index(select1)
    user_interest_vector = get_user_interest_matrix()[user_idx]
    for interest_id, is_interested in enumerate(user_interest_vector):
        if is_interested == 1:
            similar_interests = most_similar_interests_to(interest_id)
            for interest, similarity in similar_interests:
                suggestions[interest] += similarity

    # 가중치에 따라 순서대로 정렬
    suggestions = sorted(suggestions.items(),
                         key=lambda pair: pair[1],
                         reverse=True)
    
    # 이미 관심사로 설정한 요소는 역시 집계에서 제외
    if include_current_interests:
        regionresult = suggestions
        print(regionresult)
        
    else:
        regionresult = [(suggestion, weight)
                for suggestion, weight in suggestions
                if suggestion not in users_interests[select1]]
        print(regionresult)


In [12]:
# 특정 유저 클릭시 그 유저가 선택한 목록은 L3(선택한 목록)로, 선택하지 않은것만 L2(예비선택지)에 남겨두는 로직
def select_noselect():
    l2idx = 0
    l3idx = 0
    L2.delete(0, END)
    L3.delete(0, END)
    for region in unique_interests:
        if region in users_interests[select1]:
            L3.insert(l3idx, region)
            l3idx += 1
        else:
            L2.insert(l2idx, region)
            l2idx += 1

In [13]:
# # 유저를 추가하는 로직(있는 유저 추가시 유저 데이터 초기화)
# def insert_user():
#     username = E1.get()
#     users_interests[username] = []
#     L1.delete(0, END)
#     idx=0
#     for user in users_interests:
#         L1.insert(idx, user)
#         idx += 1
    

### 창부품 정리

In [14]:
# 버튼 0 >> 데이터 버튼
Bt0 = Button(root, text="데이터 불러오기", command=set_dummy)
Bt0.place(x = 10, y=10, width=100, height=30)

# 버튼 1 >> 유저베이스 결과 얻기
Bt1 = Button(root, text="유저기반 추천 받기", command=user_based_suggestions)
Bt1.place(x=10, y=260, width=120, height=30)
# 버튼 2 >> 아이템베이스 결과 얻기
Bt2 = Button(root, text="아이템기반 추천 받기", command=item_based_suggestions)
Bt2.place(x=150, y=260, width=120, height=30)

# # 버튼 4 >> 아이템 등록 버튼
# Bt4 = Button(root, text=">>", command=add_select)
# Bt4.place(x=250, y=130, width=40, height=30)
# # 버튼 5 >> 아이템 삭제 버튼
# Bt5 = Button(root, text="<<", command=delete_select)
# Bt5.place(x=250, y=200, width=40, height=30)

# # 버튼 3 >> 유저 정보 추가 버튼
# Bt3 = Button(root, text="유저 추가하기", command=insert_user)
# Bt3.place(x=150, y=300, width=120, height=30)
# # 버튼 6 >> 유저 정보 삭제 버튼
# Bt6 = Button(root, text="유저 삭제하기", command=delete_user)
# Bt6.place(x=290, y=300, width=120, height=30)

In [15]:
# 리스트박스1 >> 유저목록 보여주기
L1 = Listbox(root)

# 버튼클릭 감지 
L1.bind('<<ListboxSelect>>', onselect1)
L1.place(x=10, y=100, width=100, height=150)

# 리스트박스2 >> 아이템 목록 보여주기
L2 = Listbox(root)
L2.place(x=160, y=100, width=100, height=150)

# 버튼클릭 감지 
L2.bind('<<ListboxSelect>>', onselect2)


# 리스트박스3 >> 선택된 아이템 보여주기
L3 = Listbox(root)
L3.place(x=300, y=100, width=100, height=150)

# # 리스트박스 4 >> 추천 아이템 결과 보여주기
# L4=Listbox(root)
# L4.place(x=10, y= 360, width = 390, height= 300)

# 버튼클릭 감지
L3.bind('<<ListboxSelect>>', onselect3)

'2402010718152onselect3'

In [16]:
# # 엔트리1 >> 새 유저 추가시 사용
# E1 = Entry(root)
# E1.place(x=10, y=300, width=120, height=30)

In [17]:
# 레이블 1 >> 유저목록 리스트박스 위에
Lb1 = Label(root, text="등록유저목록")
Lb1.place(x=10, y=70, width=100, height=30)

# 레이블 2 >> 아이템 목록 리스트 위에
Lb2 = Label(root, text="아이템 목록")
Lb2.place(x=150, y=70, width=100, height=30)

# 레이블 3 >> 선택된 아이템분류 보여주기
Lb3 = Label(root, text="선택아이템목록")
Lb3.place(x=300, y=70, width=100, height=30)

# # 레이블 4 >> 결과창 보여주기
# Lb4 = Label(root, text="추천아이템목록")
# # E1.place(x=10, y=300, width=120, height=30)
# Lb4.place(x=150, y=300, width=100, height=50)

# # 레이블 5 >> 추천 아이템 결과창 보여주기
# Lb5 = Label(root, text=)
# Lb5.place(x=10, y=330, width=100, height=150)

# # 레이블 5 >> 추가
# Lb5 = Label(root, text="추가")
# Lb5.place(x=245, y=90, width=50, height=50)
# # 레이블 6 >> 삭제
# Lb6 = Label(root, text="삭제")
# Lb6.place(x=245, y=160, width=50, height=50)

In [18]:
root.mainloop()

a
과일채소


Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\tkinter\__init__.py", line 1705, in __call__
    return self.func(*args)
  File "<ipython-input-8-e428b5afd488>", line 7, in onselect1
    value = w.get(index)
  File "C:\ProgramData\Anaconda3\lib\tkinter\__init__.py", line 2798, in get
    return self.tk.call(self._w, 'get', first)
_tkinter.TclError: bad listbox index "": must be active, anchor, end, @x,y, or a number


a


Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\tkinter\__init__.py", line 1705, in __call__
    return self.func(*args)
  File "<ipython-input-8-e428b5afd488>", line 17, in onselect2
    value = w.get(index)
  File "C:\ProgramData\Anaconda3\lib\tkinter\__init__.py", line 2798, in get
    return self.tk.call(self._w, 'get', first)
_tkinter.TclError: bad listbox index "": must be active, anchor, end, @x,y, or a number


[('주먹밥', 1.2829728844144745), ('빵', 0.9342440651366262), ('김밥', 0.8848376776104931), ('생수', 0.7559289460184544), ('레토르트류', 0.7160261749006338), ('즉석조리식품', 0.5469953239549306), ('디저트', 0.47763755086988713), ('떡', 0.3086066999241838), ('두유', 0.3086066999241838), ('샐러드', 0.1889822365046136), ('햄버거류', 0.1889822365046136), ('과일채소', 0.1690308509457033)]
[('주먹밥', 2.0931935022635355), ('레토르트류', 1.2471314257997377), ('즉석조리식품', 1.196923425058676), ('떡', 1.1543203766865053), ('빵', 1.1249445384818684), ('디저트', 1.1049029006116509), ('김밥', 1.0813761376869486), ('생수', 1.0712514193323028), ('두유', 0.816227766016838), ('과일채소', 0.4082482904638631), ('햄버거류', 0.4082482904638631), ('샐러드', 0.2886751345948129)]
c
[('요구르트', 0.7731597389607807), ('빵', 0.6439505508593789), ('우유', 0.5908224762989185), ('디저트', 0.5159075191683886), ('샐러드', 0.4082482904638631), ('도시락', 0.4082482904638631), ('떡', 0.3333333333333333), ('치즈', 0.3333333333333333), ('두부', 0.20412414523193154), ('햄버거류', 0.20412414523193154), ('레토르트류', 0.2